In [43]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, Trainer, TrainingArguments
import torch
from datasets import load_dataset

import pandas as pd
import numpy as np

model_name = 'google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [47]:
from datasets import load_dataset
dataset = load_dataset("knkarthick/dialogsum")
tokenizer = AutoTokenizer.from_pretrained(model_name)

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [48]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example['dialogue']]
    example['input_ids'] = tokenizer(
        prompt, padding="max_length", truncation=True, return_tensors="pt"
    ).input_ids
    example["labels"] = tokenizer(
        example["summary"], padding="max_length", truncation=True, return_tensors="pt"
    )

In [ ]:
import pandas as pd

In [16]:
df = pd.DataFrame(dataset["train"]).head()
df = df.explode("messages")
df["raw_message"] = df.apply(lambda x: x["messages"].get("content"), axis=1)

In [20]:
df = df.loc[df["prompt"] != df["raw_message"]]

In [22]:
df["messages"][1]

{'content': "Dear Future Self,\n\nI hope you're happy and proud of what you've achieved. As I write this, I'm excited to think about our goals and how far you've come. One goal was to be a machine learning engineer. I hope you've worked hard and become skilled in this field. Keep learning and innovating. Traveling was important to us. I hope you've seen different places and enjoyed the beauty of our world. Remember the memories and lessons. Starting a family mattered to us. If you have kids, treasure every moment. Be patient, loving, and grateful for your family.\n\nTake care of yourself. Rest, reflect, and cherish the time you spend with loved ones. Remember your dreams and celebrate what you've achieved. Your determination brought you here. I'm excited to see the person you've become, the impact you've made, and the love and joy in your life. Embrace opportunities and keep dreaming big.\n\nWith love,\nKyra",
 'role': 'assistant'}

In [25]:
df["role"] = df.apply(lambda x: x["messages"].get("role"), axis=1)

/tmp/ipykernel_7517/2420875227.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["role"] = df.apply(lambda x: x["messages"].get("role"), axis=1)


In [26]:
df

,prompt,prompt_id,messages,category,raw_message,role
0,Please summarize the goals for scientists in t...,627a77298cf96a309aa35a62207c4164e22a66f6db7911...,{'content': 'Scientists are studying nests hop...,Summarize,Scientists are studying nests hoping to learn ...,assistant
1,Help write a letter of 100 -200 words to my fu...,7d443ef2cc3e34d9dc6ffcdf748c1d2a9880cd48be9c98...,"{'content': 'Dear Future Self, I hope you're ...",Generation,"Dear Future Self,\n\nI hope you're happy and p...",assistant
2,"Write a news style post about a fake event, li...",3c975b349494dea76dbbb9c01a2bb925a248efb8ca0944...,{'content': 'Today marks a new day in not only...,Generation,Today marks a new day in not only American his...,assistant
3,"Write a funny, short story about someone who w...",16d804af359db7823c457b7d82809eddaad9a5ea3c91ef...,"{'content': 'Garry has a real green thumb, and...",Generation,"Garry has a real green thumb, and taking care ...",assistant
4,Write a letter to the Editor responding to the...,e9da2fa3a6d496c5a5ee500e58e5477362698aaa08e74c...,"{'content': 'Dear Editor, Many thanks for rai...",Rewrite,"Dear Editor,\n\nMany thanks for raising this i...",assistant


In [29]:
from transformers import AutoModelForSeq2SeqLM
import torch
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):